In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import SimpleITK as sitk
import mahotas as mt
import collageradiomics

from matplotlib.patches import Rectangle
from scipy import linalg
from skimage.util.shape import view_as_windows
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import product
from skimage.feature.texture import greycomatrix

import os
import configparser

In [ ]:
# Read image
img_s = sitk.ReadImage('../sample_data/ImageSlice.png')
img_array = sitk.GetArrayFromImage(img_s)

In [ ]:
# Create random window.
svd_radius = 5
patch_window_width = 30
patch_window_height = 30
mask_min_x = 252
mask_min_y = 193
mask_max_x = mask_min_x + patch_window_width
mask_max_y = mask_min_y + patch_window_height

In [ ]:
# Read image
img_s = sitk.ReadImage('../sample_data/ImageSlice.png')
img_array = sitk.GetArrayFromImage(img_s)

In [ ]:
collage = collageradiomics.Collage.from_rectangle(img_array, mask_min_x, mask_min_y, patch_window_width, patch_window_height, svd_radius, verbose_logging=True)

In [ ]:
haralick_features = collage.execute()

In [ ]:
# Display patch
print(collageradiomics.highlight_rectangle_on_image(img_array, mask_min_x, mask_min_y, patch_window_width, patch_window_height).shape)

In [ ]:
# Display gradient
figure, axes = plt.subplots(1, 2, figsize=(15, 15))
collageradiomics.show_colored_image(figure, axes[0], collage.dx)
axes[0].set_title(f'Gx size={collage.dx.shape}')
collageradiomics.show_colored_image(figure, axes[1], collage.dy)
axes[1].set_title(f'Gy size={collage.dy.shape}')

In [ ]:
# Display dominant angles
figure, axes = plt.subplots(1, 2, figsize=(15, 15))
collageradiomics.show_colored_image(figure, axes[0], collage.dominant_angles_array)
axes[0].set_title('Dominant Angles (SVD)')
collageradiomics.show_colored_image(figure, axes[1], collage.dominant_angles_shaped)
axes[1].set_title('Dominant Angles (Color Binned)')

In [ ]:
# Display dominant angles color-binned
figure, axis = plt.subplots(1,1, figsize=(5,5))
collageradiomics.show_colored_image(figure, axis, collage.dominant_angles_shaped)

In [ ]:
# Display haralick
figure, axes = plt.subplots(3, 5, figsize=(15,15))

for row in range(3):
    for col in range(5):
        feature = row*5+col
        axis = axes[row][col]
        axis.set_axis_off()
        if feature>=13:
            continue
        collageradiomics.show_colored_image(figure, axis, haralick_features[:,:,feature])
        axis.set_title(f'Har {feature+1}')

In [ ]:
import PIL
results_folder = '../Baseline/matlab/results'
print(haralick_features[:,:,0].astype(np.float32).dtype)
print(img_array[:,:,0].dtype)
temp = haralick_features[:,:,0]
rescaled = (255.0 / temp.max() * (temp - temp.min())).astype(np.uint8)
PIL.Image.fromarray(rescaled).save(os.path.join(results_folder, 'python1.png'))

#temp = (temp - old_min) / (old_max - old_min)
#temp = temp * (new_max - new_min) + new_min
#temp = np.round(temp)
#temp = temp.astype(np.uint8)
#png.from_array(temp).save(os.path.join(results_folder, 'python1.png'))
print(temp.dtype)
#imsave(os.path.join(results_folder, 'python1.tiff'), temp.astype(np.uint16))
#temp = imread(os.path.join(results_folder, 'octave1.tiff'))
print(temp)
#print(temp.dtype)
#print(temp.shape)

In [ ]:
# save to disk

print(haralick_features.shape)
results_folder = '../Baseline/matlab/results'
subfolders = os.listdir(results_folder)
config = configparser.ConfigParser()
for subfolder in subfolders:
    
    # load config
    full_subfolder = os.path.join(results_folder, subfolder)
    print(full_subfolder)
    parameters_file = os.path.join(full_subfolder, 'parameters.cfg')
    config.read(parameters_file)
    parameters = config['DEFAULT']
    
    # load parameters
    svd_radius          = parameters.getint('svd_radius'         )
    patch_window_width  = parameters.getint('patch_window_width' )
    patch_window_height = parameters.getint('patch_window_height')
    mask_min_x          = parameters.getint('mask_min_x'         )
    mask_min_y          = parameters.getint('mask_min_y'         )
    
    # display parameters
    print(f'svd_radius = {svd_radius}')
    print(f'patch_window_width = {patch_window_width}')
    print(f'patch_window_height = {patch_window_height}')
    print(f'mask_min_x = {mask_min_x}')
    print(f'mask_min_y = {mask_min_y}')
    
    mask_max_x = mask_min_x + patch_window_width
    mask_max_y = mask_min_y + patch_window_height
    
    # run haralick
    print('Running collage:', flush=True)
    collage = collageradiomics.Collage(img_array, mask_min_x, mask_min_y, mask_max_x, mask_max_y, patch_window_width, patch_window_height, svd_radius, verbose_logging=False)
    haralick_features = collage.execute()
    print('Running collage done.', flush=True)

    # save as tif
    for f in range(13):
        print(f'Saving feature {f}:')
        temp = haralick_features[:,:,f]
        rescaled = (255.0 / temp.max() * (temp - temp.min())).astype(np.uint8)
        PIL.Image.fromarray(rescaled).save(os.path.join(full_subfolder, f'python{f+1}.png'))
        print('Saving done.')
    
print('Done!')